In [1]:
import re
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem.snowball import FrenchStemmer

In [2]:
#importation de la donnée
donnees = pd.read_csv("en.openfoodfacts.org.products.csv", delimiter="\t")
df = donnees.copy()

C:\Users\namat\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (0,24,25,26,28,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.shape

(951409, 175)

In [4]:
#pourcentatge de valeurs manquantes
def percent_missed_values(data=df):
    somme = 0
    for f in list(data.columns):
        somme += data[f].isna().sum()
    return (somme / (data.shape[0]*data.shape[1]))*100

In [5]:
percent_missed_values()

78.99605982885835

In [6]:
#Selectionner les variables se terminant par _100g
def endswith_100g(data = df):
    features_100g = []
    for f in data.columns:
        if f.endswith('_100g'):
            features_100g.append(f)
    return features_100g

In [7]:
#Conserver les variables qui ont un sens à remplacer à zéro
features_zero_100g = endswith_100g()
features_zero_100g.remove("ph_100g")
features_zero_100g.remove("nutrition-score-fr_100g")
features_zero_100g.remove("nutrition-score-uk_100g")

In [8]:
#Remplacer les valeurs manquantes par des zeros pour les variables ayant un sens
def miss_by_0(features, data=df):
    for f in features:
        data[f].fillna(0, inplace=True)

In [9]:
miss_by_0(features_zero_100g)

In [10]:
#Rammener à 100 les variables concernées
def check_100(val):
    if(val > 100):
        return 100
    return val

In [11]:
def great_to_100(features, data=df):
    for f in features:
        data[f] = data[f].apply(check_100)

In [12]:
#features conservées pour la somme à 100 
features_sum_100g = ['fat_100g', 'saturated-fat_100g', 'monounsaturated-fat_100g', 'polyunsaturated-fat_100g',
                     'omega-3-fat_100g', 'omega-6-fat_100g', 'omega-9-fat_100g', 'trans-fat_100g', 'cholesterol_100g',
                     'carbohydrates_100g', 'sugars_100g', 'starch_100g', 'polyols_100g', 'fiber_100g', 'proteins_100g',
                     'casein_100g', 'serum-proteins_100g', 'nucleotides_100g', 'salt_100g', 'alcohol_100g', 
                     'vitamin-a_100g', 'beta-carotene_100g', 'vitamin-d_100g', 'vitamin-e_100g', 'vitamin-k_100g',
                     'vitamin-c_100g', 'vitamin-b1_100g', 'vitamin-b2_100g', 'vitamin-pp_100g', 'vitamin-b6_100g',
                     'vitamin-b9_100g', 'folates_100g', 'vitamin-b12_100g', 'biotin_100g', 'pantothenic-acid_100g',
                     'silica_100g', 'bicarbonate_100g', 'potassium_100g', 'chloride_100g', 'calcium_100g', 'phosphorus_100g',
                     'iron_100g', 'magnesium_100g', 'zinc_100g', 'copper_100g', 'manganese_100g', 'fluoride_100g',
                     'selenium_100g', 'chromium_100g', 'molybdenum_100g', 'iodine_100g', 'caffeine_100g', 'taurine_100g', 
                     'fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-dried_100g', 'fruits-vegetables-nuts-estimate_100g',
                     'collagen-meat-protein-ratio_100g', 'cocoa_100g', 'chlorophyl_100g', 'carbon-footprint_100g',
                     'carbon-footprint-from-meat-or-fish_100g', 'glycemic-index_100g', 'choline_100g', 'phylloquinone_100g',
                     'beta-glucan_100g', 'inositol_100g', 'carnitine_100g']

In [13]:
df.drop(df[df[features_sum_100g].sum(axis=1) > 100].index, inplace=True)

In [14]:
df.shape

(708960, 175)

In [15]:
#supprimer les méta-données
df.drop(['created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime'], axis=1, inplace=True)

In [16]:
708960 / 951409

0.7451684816939929

In [17]:
#features retenues
features_ret = ['code','product_name', 'brands','ingredients_text', 'allergens','additives',
                'nutrition_grade_fr', 'energy_100g','fat_100g','saturated-fat_100g','sugars_100g',
                'fiber_100g','proteins_100g','salt_100g','fruits-vegetables-nuts_100g','nova_group']

In [18]:
df = df[features_ret]

In [19]:
df.shape

(708960, 16)

In [20]:
#supprimer les variables inutiles
def del_var_useless(data, seuil=1):
    var_useless = []
    for f in list(data.columns):
        if (data[f].isna().sum()/len(data)) > seuil:
            data.drop(f, axis=1, inplace=True)
            var_useless.append(f)
    print("Colonnes supprimées {}".format(var_useless))

In [21]:
del_var_useless(df, 0.9)

Colonnes supprimées ['allergens', 'additives']


C:\Users\namat\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [22]:
#supprimer des valeurs nutritionnelles négatives
def del_rows_val_neg(data=df):
    size = data.shape[0]
    fs_pos = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'sugars_100g', 'fiber_100g', 
              'proteins_100g', 'salt_100g','fruits-vegetables-nuts_100g']
    for f in fs_pos:
        data.drop(data.loc[data[f] < 0].index, inplace=True)
    dif = size -len(data)
    print("Nombre de ligne supprimer: {}, nombre de lignes restantes : {}".format(dif, data.shape[0]))

In [23]:
del_rows_val_neg(df)

Nombre de ligne supprimer: 24, nombre de lignes restantes : 708936


In [24]:
df.shape

(708936, 14)

In [25]:
df['nutrition_grade_fr'].unique()

array([nan, 'b', 'd', 'a', 'c', 'e'], dtype=object)

In [26]:
df['nova_group'].unique()

array([nan,  4.,  1.,  3.,  2.])

In [27]:
#conserver dans le meilleur des cas celui qui possède son nutriscore
def delete_doublon_better(data=df):
    size = len(data)
    index = list(data['code'].value_counts().index)
    valeurs = list(data['code'].value_counts())
    
    list_sup_one = []#liste des index de code supérieur à 1
    for i in range(0, len(index)):
        if valeurs[i] > 1:
            list_sup_one.append(index[i])
    nb_nutri_sauv = 0   
    for index in list_sup_one:#parcours de chaque index pour sauvegarder ceux qui possède l'info nutrigrade
        count = 0
        nb_with_na = len(df[(df['code'] == index) & (df['nutrition_grade_fr'].isna())])
        nb_code = len(df[df['code'] == index])
        if nb_code != nb_with_na:
            df.drop(labels=df[(df['code'] == index) & (df['nutrition_grade_fr'].isna())].index, axis=0)
            nb_nutri_sauv += 1 
    data.drop_duplicates(subset=['code'], inplace=True)
    dif = size -len(data)
    print("Nombres de doublons supprimés : {}, nombres de lignes restantes : {}".format(dif, data.shape[0]))
    print("Nombres de nutriscores sauvegardés : {}".format(nb_nutri_sauv))

In [28]:
delete_doublon_better()

Nombres de doublons supprimés : 369, nombres de lignes restantes : 708567
Nombres de nutriscores sauvegardés : 75


C:\Users\namat\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
df['nutrition_grade_fr'].isna().sum()

517896

In [30]:
df["product"] = df["product_name"]

C:\Users\namat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
#supprimer les ponctuations, les accents et les chiffres 
def del_ponct(val):
    if type(val) == str:# éviter les nan
        val = val.lower()
        val = re.compile('[éèêë]+').sub("e", val)
        val = re.compile('[àâä]+').sub("a", val)
        val = re.compile('[ùûü]+').sub("u", val)
        val = re.compile('[îï]+').sub("i", val)
        val = re.compile('[ôö]+').sub("o", val)
        return re.compile('[^A-Za-z" "]+').sub("", val)
    return val
        
def data_text_del_ponct(data=df):
    listCol = list(data.columns)
    listCol.remove("code")
    listCol.remove("product")
    for f in listCol:
        if (data[f].dtype == "object"):
            data[f] = data[f].apply(del_ponct)

In [32]:
data_text_del_ponct(df)

C:\Users\namat\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
df

,code,product_name,brands,ingredients_text,nutrition_grade_fr,energy_100g,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,fruits-vegetables-nuts_100g,nova_group,product
1,0000000000031,cacao,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,NaN,Cacao
2,00000000001111111111,sfiudwx,watt,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,NaN,Sfiudwx
3,0000000000123,sauce sweety chili,NaN,NaN,NaN,88.0,0.0,0.0,0.4,0.0,0.2,2.040,0.0,NaN,Sauce Sweety chili 0%
4,0000000000178,mini coco,NaN,NaN,NaN,251.0,3.0,1.0,3.0,0.0,2.0,1.150,0.0,NaN,Mini coco
5,0000000000208,pistou dail des ours,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,NaN,Pistou d'ail des ours
6,0000000000284,pain mais,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,NaN,Pain maïs
7,0000000000291,mendiants,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,NaN,Mendiants
8,0000000000949,salade de carottes rapees,NaN,NaN,b,134.0,0.3,0.1,3.9,0.0,0.9,0.420,0.0,NaN,Salade de carottes râpées
9,0000000000970,fromage blanc aux myrtilles,NaN,NaN,NaN,540.0,4.9,3.1,16.3,0.0,4.4,0.250,0.0,NaN,Fromage blanc aux myrtilles
10,0000000001001,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,NaN,NaN


In [34]:
#lemmatisation snowball
stemmer = FrenchStemmer(ignore_stopwords=True)
def stem(expr):
    words_stems = []
    if type(expr) == str:#eviter les nan
        expr_words = nltk.word_tokenize(expr)
        for word in expr_words:
            words_stems.append(stemmer.stem(word))
        return " ".join(words_stems)
    return expr

In [35]:
def data_text_lemma(data=df):
    listCol = list(data.columns)
    listCol.remove("code")
    listCol.remove("product")
    for f in listCol:
        if (data[f].dtype == "object"):
            data[f] = data[f].apply(stem)

In [36]:
data_text_lemma()

C:\Users\namat\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [37]:
df

,code,product_name,brands,ingredients_text,nutrition_grade_fr,energy_100g,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,fruits-vegetables-nuts_100g,nova_group,product
1,0000000000031,cacao,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,NaN,Cacao
2,00000000001111111111,sfiudwx,watt,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,NaN,Sfiudwx
3,0000000000123,sauc sweety chil,NaN,NaN,NaN,88.0,0.0,0.0,0.4,0.0,0.2,2.040,0.0,NaN,Sauce Sweety chili 0%
4,0000000000178,min coco,NaN,NaN,NaN,251.0,3.0,1.0,3.0,0.0,2.0,1.150,0.0,NaN,Mini coco
5,0000000000208,pistou dail des our,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,NaN,Pistou d'ail des ours
6,0000000000284,pain mais,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,NaN,Pain maïs
7,0000000000291,mendi,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,NaN,Mendiants
8,0000000000949,salad de carott rape,NaN,NaN,b,134.0,0.3,0.1,3.9,0.0,0.9,0.420,0.0,NaN,Salade de carottes râpées
9,0000000000970,fromag blanc aux myrtill,NaN,NaN,NaN,540.0,4.9,3.1,16.3,0.0,4.4,0.250,0.0,NaN,Fromage blanc aux myrtilles
10,0000000001001,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,NaN,NaN


In [38]:
features_to_empty = ['product_name', 'brands', 'ingredients_text', "product"]
def miss_by_empty(features, data=df):
    for f in features:
        data[f].fillna(" ", inplace=True)
        
miss_by_empty(features_to_empty)

C:\Users\namat\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [39]:
df[['product_name', 'brands', 'ingredients_text', "product"]]

,product_name,brands,ingredients_text,product
1,cacao,,,Cacao
2,sfiudwx,watt,,Sfiudwx
3,sauc sweety chil,,,Sauce Sweety chili 0%
4,min coco,,,Mini coco
5,pistou dail des our,,,Pistou d'ail des ours
6,pain mais,,,Pain maïs
7,mendi,,,Mendiants
8,salad de carott rape,,,Salade de carottes râpées
9,fromag blanc aux myrtill,,,Fromage blanc aux myrtilles
10,,,,


In [40]:
df.to_csv("nettoyage.csv", sep="\t", index=False)